# 布林拐头策略

In [1]:
# 数据接口 
import akshare as ak
import baostock as bs
import tushare as ts

# 基础模块
import datetime as dt
from itertools import product
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
import time

# 回测框架
import backtrader as bt
import backtrader.indicators as btind
import backtrader.feeds as btfeed

# 基础函数
import utilsJ

## 策略主体

买入信号：波浪均线出现向上拐头（前两日均下降，今日上升）且当日收盘价在布林带中线之上。

买入仓位：根据海龟ATR仓位计算最优仓位。

卖出信号：波浪均线出现向下拐头（前两日均上升，今日下降）或收盘价向下突破布林带上界。

卖出仓位：一律平仓。

### 个股版本

In [2]:
class BW_s(bt.Strategy):
    
    params = (
        # General params
        ('printlog', False),
        ('units', 1),
        ('p_stake', 100),

        # Indicator params
        ('bollinger_per', 20),
        ('bollinger_dev', 2),
        ('direction_lag', 5),
        ('wave_period', 7),
        
        # ATR params
        ('using_atr', False),
        ('atr_period', 14),
        ('atr_percent', 1),
    )
    
    
    def log(self, txt, dt=None, doprint=False):
        if self.params.printlog or doprint:
            dt = dt or self.datas[0].datetime.date(0)
            print('%s: %s' % (dt.isoformat(), txt))
            #with open('log.txt', 'a') as file:
                #file.write('%s: %s \n' % (dt.isoformat(), txt))
        
    
    def __init__(self):

        # Initialization
        self.buyprice = 0
        self.sellprice = 0
        self.order = None

        # Alias
        self.dataclose = self.datas[0].close
        self.datahigh = self.datas[0].high
        self.datalow = self.datas[0].low

        # Indicators
        ## Bollinger
        self.bollinger = btind.BollingerBands(self.dataclose, 
                                              period=self.p.bollinger_per, 
                                              devfactor=self.p.bollinger_dev)

        ## Wave indicator
        self.sma = btind.SMA(self.dataclose, period=self.p.wave_period)

        ## ATR indicator
        self.atr_initial = self.broker.get_cash()
        self.tr = btind.Max((self.datahigh - self.datalow), 
                            abs(self.dataclose(-1) - self.datahigh),
                            abs(self.dataclose(-1) - self.datalow))
        self.atr = btind.SMA(self.tr, period=self.p.atr_period)

        # Signals
        self.bollinger_sell = bt.And(self.dataclose(0) < self.bollinger.top(0),
                                     self.dataclose(-1) > self.bollinger.top(-1))
        self.wave_buy = bt.And(self.sma(0) > self.sma(-1), self.sma(-1) < self.sma(-2), 
                               self.sma(-2) < self.sma(-3), self.sma(0) > self.bollinger.mid(0))
        self.wave_sell = bt.And(self.sma(0) < self.sma(-1), self.sma(-1) > self.sma(-2), 
                                self.sma(-2) > self.sma(-3))


    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log('BUY EXECUTED, Price: %.2f, Lot:%i, Position:%i, Cash: %i, Value: %i' %
                         (order.executed.price, order.executed.size,
                          self.getposition(self.data).size,
                          self.broker.get_cash(), self.broker.get_value()))
                self.buyprice = order.executed.price
            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Lot:%i, Position:%i, Cash: %i, Value: %i' %
                         (order.executed.price, -order.executed.size,
                          self.getposition(self.data).size,
                          self.broker.get_cash(), self.broker.get_value()))
                self.sellprice = order.executed.price

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        # Write down: no pending order
        self.order = None



    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))

    def next(self):
        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return
        
        buy_s = self.wave_buy[0]
        sell_s = self.bollinger_sell[0] or self.wave_sell[0]
        
        if buy_s and self.getposition(self.data).size == 0:
            # ATR position
            if self.p.using_atr:
                opt_pos = np.round(self.atr_initial * self.p.atr_percent / self.p.p_stake / self.atr[0])
                if opt_pos*self.p.p_stake > 0 and self.getposition(self.data).size < opt_pos*self.p.p_stake:
                    self.log('BUY CREATE, Price: %.2f, Lots: %i, Current Position: %i' % 
                             (self.dataclose[0], opt_pos*self.p.p_stake-self.getposition(self.data).size,
                             self.getposition(self.data).size))
                    self.order = self.buy(size=opt_pos*self.p.p_stake-self.getposition(self.data).size)
            # Normal position
            else:
                cash_ava = self.broker.get_cash() / self.p.units
                lots = np.floor(cash_ava / (self.p.p_stake*self.dataclose[0]))*self.p.p_stake
                self.log('BUY CREATE, Price: %.2f, Lots: %i, Current Position: %i' % 
                         (self.dataclose[0], lots, self.getposition(self.data).size))
                self.order = self.buy(size=lots)
        elif sell_s and self.getposition(self.data).size > 0:
            self.log('Sell CREATE(Close), Price: %.2f, Current Position: %i' % 
                     (self.dataclose[0], self.getposition(self.data).size))
            self.order = self.close()

    
    #def stop(self):
    #    self.log('Ending Position:%i. Ending Value:%.2f.' %
    #            (self.getposition(self.data).size,
    #            self.cerebro.broker.getvalue()), doprint=False)

### 多股版本

In [ ]:
class BW_m(bt.Strategy):
    
    params = (
        # General params
        ('printlog', False),
        ('units', 300),
        ('p_stake', 100),

        # Indicator params
        ('bollinger_per', 20),
        ('bollinger_dev', 2),
        ('wave_period_buy', 7),
        ('wave_period_sell', 7),

        # ATR params
        ('using_atr', False),
        ('atr_period', 14),
        ('atr_percent', 1),
    )

    
    def log(self, txt, dt=None, doprint=False):
        if self.params.printlog or doprint:
            dt = dt or self.datas[0].datetime.date(0)
            print('%s: %s' % (dt.isoformat(), txt))
            #with open('log.txt', 'a') as file:
                #file.write('%s: %s \n' % (dt.isoformat(), txt))

    
    def __init__(self):

        # Global initialization
        self.atr_initial = self.broker.get_cash()
        self.inds = dict()
        #self.profit_i = dict()
        self.cash_unit = self.broker.get_cash() / self.params.units


        for d in self.datas:

            # Local initialization
            self.inds[d] = dict()
            self.inds[d]['buyprice'] = 0
            self.inds[d]['sellprice'] = 0
            self.inds[d]['order'] = None

            # Alias
            self.inds[d]['dataclose'] = d.close
            self.inds[d]['datahigh'] = d.high
            self.inds[d]['datalow'] = d.low

            # Indicators
            ## Bollinger Bands
            self.inds[d]['bollinger'] = btind.BollingerBands(self.inds[d]['dataclose'], 
                                                             period=self.p.bollinger_per, 
                                                             devfactor=self.p.bollinger_dev)
            self.inds[d]['bollinger_buy'] = bt.And(self.inds[d]['dataclose'](0) > self.inds[d]['bollinger'].mid(0), 
                                                   self.inds[d]['dataclose'](-1) < self.inds[d]['bollinger'].mid(-1))

            self.inds[d]['bollinger_sell'] = bt.And(self.inds[d]['dataclose'](0) < self.inds[d]['bollinger'].top(0), 
                                                    self.inds[d]['dataclose'](-1) > self.inds[d]['bollinger'].top(-1))

            ## Wave
            self.inds[d]['sma_b'] = btind.SMA(d, period=self.p.wave_period_buy)
            self.inds[d]['sma_s'] = btind.SMA(d, period=self.p.wave_period_sell)
            self.inds[d]['wave_buy'] = bt.And(self.inds[d]['sma_b'](0) > self.inds[d]['sma_b'](-1), 
                                              self.inds[d]['sma_b'](-1) < self.inds[d]['sma_b'](-2), 
                                              self.inds[d]['sma_b'](-2) < self.inds[d]['sma_b'](-3),
                                              self.inds[d]['sma_b'](0) > self.inds[d]['bollinger'].mid(0))
            self.inds[d]['wave_sell'] = bt.And(self.inds[d]['sma_s'](0) < self.inds[d]['sma_s'](-1), 
                                               self.inds[d]['sma_s'](-1) > self.inds[d]['sma_s'](-2), 
                                               self.inds[d]['sma_s'](-2) > self.inds[d]['sma_s'](-3))
        
            ## ATR
            self.inds[d]['tr'] = btind.Max((self.inds[d]['datahigh'](0) - self.inds[d]['datalow'](0)), 
                                           abs(self.inds[d]['dataclose'](-1) - self.inds[d]['datahigh'](0)),
                                           abs(self.inds[d]['dataclose'](-1) - self.inds[d]['datalow'](0)))
            self.inds[d]['atr'] = btind.SMA(self.inds[d]['tr'], period=self.p.atr_period)


    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                if self.inds[self.data_tracker]['wave_buy']:
                    self.inds[self.data_tracker]['buyprice']
                self.log('买单执行, 代码：%s, 价格:%.2f, 股数:%i, 持有现金:%i' %
                         (order.info['name'],
                          order.executed.price,
                          order.executed.size,
                          self.broker.get_cash()))

            else:  # Sell
                self.inds[self.data_tracker]['sellprice'] = order.executed.price
                self.log('卖单执行, 代码：%s, 价格:%.2f, 股数:%i, 持有现金:%i' %
                        (order.info['name'], order.executed.price,
                         -order.executed.size, self.broker.get_cash()))

        elif order.status in [order.Canceled]:
            self.log('订单取消：被撤销')

        elif order.status in [order.Margin]:
            self.log('订单取消：资金不足，代码：%s，持有资金:%i' %
                        (order.info['name'],
                         self.broker.get_cash()))

        elif order.status in [order.Rejected]:
            self.log('订单取消：被拒绝')

        # Write down: no pending order
        #self.order = None


    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('交易收益：代码：%s, 毛利润 %.2f, 净利润 %.2f' %
                 (trade.history[0]['event']['order'].info['name'], 
                  trade.pnl, trade.pnlcomm))
        

    def next(self):

        buy_list = []
        sell_list = []

        for d in self.datas:
            # Check if an order is pending ... if yes, we cannot send a 2nd one
            if self.inds[d]['order']:
                return

            buy_indicator = self.inds[d]['wave_buy']
            sell_indicator = self.inds[d]['wave_sell'] or self.inds[d]['bollinger_sell']

            if sell_indicator and self.getposition(d).size > 0:
                sell_list.append((d, -1))

            elif buy_indicator and self.getposition(d).size == 0:
                if self.params.using_atr:
                    opt_pos = np.floor(self.atr_initial * self.params.atr_percent / self.params.p_stake / self.inds[d]['atr'][0])
                    if opt_pos*self.params.p_stake > 0 and self.getposition(d).size < opt_pos*self.params.p_stake:
                        buy_list.append((d, 
                                        opt_pos*self.params.p_stake-self.getposition(d).size, 
                                        (opt_pos*self.params.p_stake-self.getposition(d).size)*self.inds[d]['dataclose'][0]))
                else:
                    lots = np.floor(self.cash_unit / (self.params.p_stake*self.inds[d]['dataclose'][0]))*self.params.p_stake
                    buy_list.append((d, lots, lots * self.inds[d]['dataclose'][0]))
        
        for s_order in sell_list:
            self.data_tracker = s_order[0]
            self.log('卖单创建, 代码: %s, 价格: %.2f, 股数:%i, 现有持仓: %i' % 
                        (s_order[0]._name, 
                        self.inds[s_order[0]]['dataclose'][0],
                        s_order[1] if s_order[1] != -1 else self.getposition(s_order[0]).size,
                        self.getposition(s_order[0]).size))
            self.order = self.sell(data=s_order[0], 
                         size=s_order[1] if s_order[1] != -1 else self.getposition(s_order[0]).size, 
                         name=s_order[0]._name)
        
        #buy_list = sorted(buy_list, key=lambda x: x[2], reverse=False)
        random.shuffle(buy_list)
        for b_order in buy_list:
            self.data_tracker = b_order[0]
            self.log('买单创建, 代码: %s, 价格: %.2f, 股数: %i, 现有持仓: %i' % 
                        (b_order[0]._name, 
                        self.inds[b_order[0]]['dataclose'][0], 
                        b_order[1],
                        self.getposition(b_order[0]).size))
            self.order = self.buy(data=b_order[0], size=b_order[1], name=b_order[0]._name)


    #def stop(self):
        #for d in self.datas:
        #    if self.getposition(d).size > 0:
        #        self.profit_i[d._name] += self.getposition(d).size * d.close[0]
        #self.log('Ending Value:%.2f.' %
        #        (self.cerebro.broker.getvalue()), doprint=False)
        #print(sorted(self.profit_i.items(), key=lambda x: x[1]))
        #print(sum([x[1] for x in self.profit_i.items()]))
        #for d in self.datas:
        #    if self.getposition(d).size > 0:
        #        self.profit_i[d._name] += self.getposition(d).size * self.inds[d]['dataclose'][0]
        #print(sorted(self.profit_i.items(), key= lambda x:x[1]))
        #return self.profit_i

## 回测

### 个股回测

In [ ]:
stock_index = '000166.SZ'
startdate = dt.date(2020,1,1) - dt.timedelta(days=20)
endddate = dt.date(2020,12,31)


if __name__ ==  '__main__':
    
    # Initialization
    cerebro = bt.Cerebro()
    strats = cerebro.addstrategy(BW_s, printlog=True, direction_lag=20) 

    # Data
    df = utilsJ.get_stock(stock_index, startdate, endddate)
    data = btfeed.PandasData(dataname=df,fromdate=startdate,todate=endddate)
    cerebro.adddata(data)

    # Start condition
    cerebro.broker = bt.brokers.BackBroker(coc=True)   
    cerebro.broker.setcash(20000)
    #cerebro.broker.setcommission()
    start_value = cerebro.broker.getvalue()
    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

    # Execution
    cerebro.run()

    # Final result
    final_value = cerebro.broker.getvalue()
    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
    print('Net Profit: %.2f%%' % ((final_value - start_value) / start_value * 100))
    cerebro.plot(iplot=False)

### 个股沪深300回测(调参)

In [ ]:
index_code = '399300.SZ'
startdate = dt.date(2020,1,1) - dt.timedelta(days=20)
enddate = dt.date(2020,12,31)

if __name__ == '__main__':

    index_list = utilsJ.get_index_components(index_code, startdate, enddate)
    profit_stk = dict()
    for stk in index_list:
        max_profit = (0,0.0,0)
        for i in range(3,7):
            for j in np.arange(1.5,3,0.1):

                # Initialization
                cerebro = bt.Cerebro()
                strats = cerebro.addstrategy(BW_s, printlog=False, direction_lag=i, bollinger_dev=float(j)) 

                # Data
                df = utilsJ.get_stock(stk, startdate, enddate)
                data = btfeed.PandasData(dataname=df,fromdate=startdate,todate=enddate)
                cerebro.adddata(data)

                # Start condition
                cerebro.broker = bt.brokers.BackBroker(coc=True)   
                cerebro.broker.setcash(300000)
                #cerebro.broker.setcommission()
                start_value = cerebro.broker.getvalue()
                #print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

                # Execution
                cerebro.run()

                # Final result
                final_value = cerebro.broker.getvalue()
                #print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
                print('%s, (%i, %.1f) Net Profit: %.2f%%' % (stk,i,j,(final_value - start_value) / start_value * 100))
                if max_profit[2] <  (final_value-start_value)/start_value*100:
                    max_profit = (i,j,(final_value-start_value)/start_value*100)
                    
        profit_stk[stk] = (final_value - start_value) / start_value * 100

### 多股回测
注：存在着Backtrader里面数据线必须对齐保持起始日期一致的问题。因此手动将数据对齐，对于晚上市的股票，手动在前面添加了空数据行。

In [ ]:
startdate = dt.date(2020,1,1) - dt.timedelta(days=20)
enddate = dt.date(2020,12,31)
date_idx =  utilsJ.get_stock('000001.SZ', startdate, enddate).trade_date
index_code = '399300.SZ'
index_list = utilsJ.get_index_components(index_code, startdate, enddate)

if __name__ ==  '__main__':
    
    # Initialization
    cerebro = bt.Cerebro(tradehistory=True)
    strats = cerebro.addstrategy(BW_m, printlog=False, units=350)

    # Data
    for stk in index_list:
        df = utilsJ.get_stock(stk, startdate, enddate)
        df_new = pd.DataFrame([[x,0,0,0,0,0,0] for x in date_idx if x not in df.index], columns=['datetime', 'open', 'high', 'low', 'close', 'volume'])
        df_new.set_index('trade_date', inplace=True)
        if len(df_new) != 0:
            df = pd.concat([df_new, df])
        df.sort_index(inplace=True)
        data = btfeed.PandasData(dataname=df,fromdate=startdate,todate=enddate)
        cerebro.adddata(data, name=stk)

    # Start condition
    cerebro.broker = bt.brokers.BackBroker(coc=True)
    cerebro.broker.setcash(50000000)
    #cerebro.broker.setcommission()
    start_value = cerebro.broker.getvalue()
    #print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

    # Execution
    cerebro.run(maxcpus=16)

    # Final result
    final_value = cerebro.broker.getvalue()
    #print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
    print('Net Profit: %.2f%%' % ((final_value - start_value) / start_value * 100))
    #profit.append((final_value - start_value) / start_value * 100)

    #cerebro.plot(iplot=False)

### 多股循环调参

In [ ]:
profit_dict = {}
startdate = dt.date(2020, 1, 1) - dt.timedelta(days=20)
enddate = dt.date(2020, 12, 31)
date_idx =  utilsJ.get_stock('000001.SZ', startdate, enddate).trade_date

index_code = '399300.SZ'
index_list = utilsJ.get_index_components(index_code, startdate, enddate)


if __name__ == '__main__':
    for i in np.arange(1.5,3,0.1):
        
        # Initialization
        cerebro = bt.Cerebro(tradehistory=True)
        strats = cerebro.addstrategy(BW_m, printlog=False, units=len(index_list), wave_period_buy=3, wave_period_sell=6, bollinger_dev=float(i))
        
        # Data
        for stk in index_list:
            df = utilsJ.get_stock(stk, startdate, enddate)
            df_new = pd.DataFrame([[x,0,0,0,0,0,0] for x in date_idx if x not in df.index], columns=['datetime', 'open', 'high', 'low', 'close', 'volume'])
            df_new.set_index('trade_date', inplace=True)
            if len(df_new) != 0:
                df = pd.concat([df_new, df])
            df.sort_index(inplace=True)
            data = btfeed.PandasData(dataname=df,fromdate=startdate,todate=enddate)
            cerebro.adddata(data, name=stk)
        
        # Start condition
        cerebro.broker = bt.brokers.BackBroker(coc=True)
        cerebro.broker.setcash(5000 * len(index_list) * 100)
        #cerebro.broker.setcommission()
        start_value = cerebro.broker.getvalue()
        #print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

        # Execution
        cerebro.run(maxcpus=16)

        # Final result
        final_value = cerebro.broker.getvalue()
        #print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
        #print('(%s, %s) Net Profit: %.2f%%' % (s[0], s[1], (final_value-start_value)/start_value*100))
        print((i, (final_value - start_value) / start_value * 100))